In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from langchain.llms import OpenAI  # LLM for offer recommendations
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

In [ ]:
np.random.seed(123)

In [16]:
sample_size = 1000
data = {
    'Customer_ID': [f'C{i+1}' for i in range(sample_size)],
    'Monthly_Calls': np.random.normal(loc=300, scale=100, size=sample_size).astype(int),
    'Monthly_SMS': np.random.poisson(lam=30, size=sample_size),
    'Data_Usage_GB': np.random.lognormal(mean=2, sigma=0.5, size=sample_size),
    'Subscription_Type': np.random.choice(['Postpaid', 'Prepaid'], size=sample_size, p=[0.6, 0.4]),
}

data['Average_Bill'] = [np.random.normal(70,15) if sub == 'Postpaid' else np.random.normal(40,10) for sub in data['Subscription_Type'] ]

In [17]:
df = pd.DataFrame(data)

In [18]:
df

,Customer_ID,Monthly_Calls,Monthly_SMS,Data_Usage_GB,Subscription_Type,Average_Bill
0,C1,443,38,16.092193,Prepaid,37.271282
1,C2,546,27,4.297761,Prepaid,51.386492
2,C3,341,30,7.739410,Prepaid,20.332755
3,C4,424,25,5.754387,Prepaid,25.643831
4,C5,295,35,5.823838,Postpaid,92.985287
...,...,...,...,...,...,...
995,C996,74,28,3.303418,Postpaid,65.442189
996,C997,172,34,5.704035,Postpaid,68.434246
997,C998,351,28,4.934610,Postpaid,84.885176
998,C999,366,25,6.371923,Postpaid,60.227683


In [20]:
df['Monthly_Calls'] = df['Monthly_Calls'].clip(lower=0)

In [26]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Monthly_Calls,1000.0,297.212000,99.881467,0.000000,226.000000,296.000000,364.000000,629.000000
Monthly_SMS,1000.0,29.625000,5.190117,16.000000,26.000000,29.000000,33.000000,47.000000
Data_Usage_GB,1000.0,8.440099,4.196202,1.645651,5.458518,7.639541,10.514241,39.327753
Average_Bill,1000.0,59.691053,20.461590,14.168032,42.622707,59.734930,75.619664,111.915453


In [29]:
df.head()

,Customer_ID,Monthly_Calls,Monthly_SMS,Data_Usage_GB,Subscription_Type,Average_Bill
0,C1,443,38,16.092193,0,37.271282
1,C2,546,27,4.297761,0,51.386492
2,C3,341,30,7.739410,0,20.332755
3,C4,424,25,5.754387,0,25.643831
4,C5,295,35,5.823838,1,92.985287


In [28]:
df['Subscription_Type'] = df['Subscription_Type'].map({'Postpaid':1,'Prepaid':0})

In [31]:
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(df.drop(columns= ['Customer_ID']))

In [33]:
similarity_matrix = cosine_similarity(features_scaled)

In [35]:
similarity_df = pd.DataFrame(similarity_matrix, index=df['Customer_ID'], columns= df['Customer_ID'])

In [36]:
similarity_df

Customer_ID,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C991,C992,C993,C994,C995,C996,C997,C998,C999,C1000
Customer_ID,,,,,,,,,,,,,,,,,,,,,
C1,1.000000,0.880988,0.977922,0.910832,0.605865,0.470122,0.918236,0.639008,0.587765,0.994597,...,0.494646,0.516999,0.985291,0.963380,0.438707,0.378662,0.538553,0.563497,0.548643,0.515477
C2,0.880988,1.000000,0.903726,0.971601,0.616182,0.593622,0.995758,0.535402,0.535981,0.912832,...,0.522932,0.572130,0.910379,0.950279,0.473649,0.363199,0.496111,0.630824,0.609103,0.552856
C3,0.977922,0.903726,1.000000,0.955531,0.549353,0.436533,0.928173,0.553419,0.541393,0.966780,...,0.414325,0.466264,0.944209,0.933228,0.362502,0.319649,0.484102,0.523669,0.530308,0.455434
C4,0.910832,0.971601,0.955531,1.000000,0.532249,0.512760,0.971125,0.490299,0.502093,0.920121,...,0.413505,0.488952,0.906206,0.921540,0.394663,0.286194,0.434786,0.549816,0.563505,0.479425
C5,0.605865,0.616182,0.549353,0.532249,1.000000,0.931885,0.637399,0.958285,0.968982,0.640878,...,0.982596,0.982085,0.652974,0.682707,0.927279,0.953739,0.982932,0.986323,0.952617,0.964802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C996,0.378662,0.363199,0.319649,0.286194,0.953739,0.902289,0.384569,0.929611,0.960158,0.405597,...,0.961381,0.959015,0.417028,0.438762,0.928197,1.000000,0.983109,0.943694,0.926769,0.945952
C997,0.538553,0.496111,0.484102,0.434786,0.982932,0.901803,0.523392,0.965357,0.986715,0.559882,...,0.963967,0.966572,0.565751,0.581597,0.917367,0.983109,1.000000,0.962835,0.945150,0.951830
C998,0.563497,0.630824,0.523669,0.549816,0.986323,0.977257,0.639351,0.933689,0.963305,0.603529,...,0.974782,0.996276,0.618324,0.654393,0.953108,0.943694,0.962835,1.000000,0.983524,0.984440


In [37]:
similarity_df_no_self = similarity_df.apply(lambda x: x.replace(1, np.nan))

In [38]:
similarity_df_no_self

Customer_ID,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C991,C992,C993,C994,C995,C996,C997,C998,C999,C1000
Customer_ID,,,,,,,,,,,,,,,,,,,,,
C1,NaN,0.880988,0.977922,0.910832,0.605865,0.470122,0.918236,0.639008,0.587765,0.994597,...,0.494646,0.516999,0.985291,0.963380,0.438707,0.378662,0.538553,0.563497,0.548643,0.515477
C2,0.880988,1.000000,0.903726,0.971601,0.616182,0.593622,0.995758,0.535402,0.535981,0.912832,...,0.522932,0.572130,0.910379,0.950279,0.473649,0.363199,0.496111,0.630824,0.609103,0.552856
C3,0.977922,0.903726,NaN,0.955531,0.549353,0.436533,0.928173,0.553419,0.541393,0.966780,...,0.414325,0.466264,0.944209,0.933228,0.362502,0.319649,0.484102,0.523669,0.530308,0.455434
C4,0.910832,0.971601,0.955531,1.000000,0.532249,0.512760,0.971125,0.490299,0.502093,0.920121,...,0.413505,0.488952,0.906206,0.921540,0.394663,0.286194,0.434786,0.549816,0.563505,0.479425
C5,0.605865,0.616182,0.549353,0.532249,1.000000,0.931885,0.637399,0.958285,0.968982,0.640878,...,0.982596,0.982085,0.652974,0.682707,0.927279,0.953739,0.982932,0.986323,0.952617,0.964802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C996,0.378662,0.363199,0.319649,0.286194,0.953739,0.902289,0.384569,0.929611,0.960158,0.405597,...,0.961381,0.959015,0.417028,0.438762,0.928197,1.000000,0.983109,0.943694,0.926769,0.945952
C997,0.538553,0.496111,0.484102,0.434786,0.982932,0.901803,0.523392,0.965357,0.986715,0.559882,...,0.963967,0.966572,0.565751,0.581597,0.917367,0.983109,NaN,0.962835,0.945150,0.951830
C998,0.563497,0.630824,0.523669,0.549816,0.986323,0.977257,0.639351,0.933689,0.963305,0.603529,...,0.974782,0.996276,0.618324,0.654393,0.953108,0.943694,0.962835,1.000000,0.983524,0.984440


In [39]:
similarity_df.head()

Customer_ID,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C991,C992,C993,C994,C995,C996,C997,C998,C999,C1000
Customer_ID,,,,,,,,,,,,,,,,,,,,,
C1,1.000000,0.880988,0.977922,0.910832,0.605865,0.470122,0.918236,0.639008,0.587765,0.994597,...,0.494646,0.516999,0.985291,0.963380,0.438707,0.378662,0.538553,0.563497,0.548643,0.515477
C2,0.880988,1.000000,0.903726,0.971601,0.616182,0.593622,0.995758,0.535402,0.535981,0.912832,...,0.522932,0.572130,0.910379,0.950279,0.473649,0.363199,0.496111,0.630824,0.609103,0.552856
C3,0.977922,0.903726,1.000000,0.955531,0.549353,0.436533,0.928173,0.553419,0.541393,0.966780,...,0.414325,0.466264,0.944209,0.933228,0.362502,0.319649,0.484102,0.523669,0.530308,0.455434
C4,0.910832,0.971601,0.955531,1.000000,0.532249,0.512760,0.971125,0.490299,0.502093,0.920121,...,0.413505,0.488952,0.906206,0.921540,0.394663,0.286194,0.434786,0.549816,0.563505,0.479425
C5,0.605865,0.616182,0.549353,0.532249,1.000000,0.931885,0.637399,0.958285,0.968982,0.640878,...,0.982596,0.982085,0.652974,0.682707,0.927279,0.953739,0.982932,0.986323,0.952617,0.964802


In [40]:
array = similarity_df.to_numpy()

In [41]:
np.fill_diagonal(array, np.nan)

In [42]:
df_replaced = pd.DataFrame(array, columns=df.Customer_ID, index=df.Customer_ID)

In [43]:
df_replaced

Customer_ID,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C991,C992,C993,C994,C995,C996,C997,C998,C999,C1000
Customer_ID,,,,,,,,,,,,,,,,,,,,,
C1,NaN,0.880988,0.977922,0.910832,0.605865,0.470122,0.918236,0.639008,0.587765,0.994597,...,0.494646,0.516999,0.985291,0.963380,0.438707,0.378662,0.538553,0.563497,0.548643,0.515477
C2,0.880988,NaN,0.903726,0.971601,0.616182,0.593622,0.995758,0.535402,0.535981,0.912832,...,0.522932,0.572130,0.910379,0.950279,0.473649,0.363199,0.496111,0.630824,0.609103,0.552856
C3,0.977922,0.903726,NaN,0.955531,0.549353,0.436533,0.928173,0.553419,0.541393,0.966780,...,0.414325,0.466264,0.944209,0.933228,0.362502,0.319649,0.484102,0.523669,0.530308,0.455434
C4,0.910832,0.971601,0.955531,NaN,0.532249,0.512760,0.971125,0.490299,0.502093,0.920121,...,0.413505,0.488952,0.906206,0.921540,0.394663,0.286194,0.434786,0.549816,0.563505,0.479425
C5,0.605865,0.616182,0.549353,0.532249,NaN,0.931885,0.637399,0.958285,0.968982,0.640878,...,0.982596,0.982085,0.652974,0.682707,0.927279,0.953739,0.982932,0.986323,0.952617,0.964802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C996,0.378662,0.363199,0.319649,0.286194,0.953739,0.902289,0.384569,0.929611,0.960158,0.405597,...,0.961381,0.959015,0.417028,0.438762,0.928197,NaN,0.983109,0.943694,0.926769,0.945952
C997,0.538553,0.496111,0.484102,0.434786,0.982932,0.901803,0.523392,0.965357,0.986715,0.559882,...,0.963967,0.966572,0.565751,0.581597,0.917367,0.983109,NaN,0.962835,0.945150,0.951830
C998,0.563497,0.630824,0.523669,0.549816,0.986323,0.977257,0.639351,0.933689,0.963305,0.603529,...,0.974782,0.996276,0.618324,0.654393,0.953108,0.943694,0.962835,NaN,0.983524,0.984440


In [44]:
most_similar = df_replaced.idxmax(axis=1)

In [47]:
most_similar

Customer_ID
C1       C605
C2       C475
C3       C661
C4       C851
C5       C130
         ... 
C996     C467
C997     C159
C998     C839
C999     C538
C1000    C332
Length: 1000, dtype: object

In [48]:
# LLM usage

In [86]:
api_key = "gsk_"
llm=ChatGroq(groq_api_key=api_key,model_name="Gemma2-9b-It")

In [106]:
prompt_template = PromptTemplate(
                                 
input_variables=["customer_id", "similar_customer", "calls", "sms", "data", "bill", "subscription"],

template="""
    A telecom customer {customer_id} has the following usage pattern:
    - Monthly Calls: {calls} mins
    - Monthly SMS: {sms}
    - Data Usage: {data} GB
    - Average Monthly Bill: ${bill}
    - Subscription Type: {subscription}

    Their most similar customer is {similar_customer}, who has a similar usage profile.
    
    Based on this, suggest a personalized offer for customer {customer_id} that aligns with their usage behavior.
    """
    
)

In [108]:
def generate_offer(customer):
    similar_customer = most_similar[customer]
    
    # Get feature values of the customer
    customer_data = df[df['Customer_ID'] == customer]
    if customer_data.empty:
        return 'error: customer not found'
    customer_data= customer_data.iloc[0]
    print(f"Generating offer for Customer: {customer}")
    print(f"Similar Customer: {similar_customer}")
    print(f"Customer Data: {customer_data.to_dict()}")
    
    # Fill the prompt
    prompt = prompt_template.format(
        customer_id=customer,
        similar_customer=similar_customer,
        calls=customer_data['Monthly_Calls'],
        sms=customer_data['Monthly_SMS'],
        data=round(customer_data['Data_Usage_GB'], 2),
        bill=round(customer_data['Average_Bill'], 2),
        subscription="Postpaid" if customer_data['Subscription_Type'] == 1 else "Prepaid"

    )

    print(f"Prompt for {customer}: {prompt}")
  
    #offer = llm(prompt)
    #return offer
    if not callable(llm):
        raise TypeError("llm is not callable. Check initialization.")

    try:
        offer = llm(prompt)
        print(f"Offer for {customer}: {offer}")  # Debugging output
    except Exception as e:
        return f"LLM Error: {e}"

    # Debugging: Ensure output is a string
    if not isinstance(offer, str):
        return f"Error: Unexpected return type from LLM - {type(offer)}"

    return offer
  




    #offer = llm(prompt)
    #return offer

In [109]:
df['Recommended_Offer'] = df['Customer_ID'].apply(lambda x: generate_offer(x))

Generating offer for Customer: C1
Similar Customer: C605
Customer Data: {'Customer_ID': 'C1', 'Monthly_Calls': 443, 'Monthly_SMS': 38, 'Data_Usage_GB': 16.0921932476633, 'Subscription_Type': 0, 'Average_Bill': 37.27128243756963, 'Recommended_Offer': 'LLM Error: Got unknown type \n'}
Prompt for C1: 
    A telecom customer C1 has the following usage pattern:
    - Monthly Calls: 443 mins
    - Monthly SMS: 38
    - Data Usage: 16.09 GB
    - Average Monthly Bill: $37.27
    - Subscription Type: Prepaid

    Their most similar customer is C605, who has a similar usage profile.
    
    Based on this, suggest a personalized offer for customer C1 that aligns with their usage behavior.
    
Generating offer for Customer: C2
Similar Customer: C475
Customer Data: {'Customer_ID': 'C2', 'Monthly_Calls': 546, 'Monthly_SMS': 27, 'Data_Usage_GB': 4.297761334848371, 'Subscription_Type': 0, 'Average_Bill': 51.38649173053231, 'Recommended_Offer': 'LLM Error: Got unknown type \n'}
Prompt for C2: 
    A

In [113]:
df.head()

,Customer_ID,Monthly_Calls,Monthly_SMS,Data_Usage_GB,Subscription_Type,Average_Bill,Recommended_Offer
0,C1,443,38,16.092193,0,37.271282,LLM Error: Got unknown type \n
1,C2,546,27,4.297761,0,51.386492,LLM Error: Got unknown type \n
2,C3,341,30,7.739410,0,20.332755,LLM Error: Got unknown type \n
3,C4,424,25,5.754387,0,25.643831,LLM Error: Got unknown type \n
4,C5,295,35,5.823838,1,92.985287,LLM Error: Got unknown type \n
